# Bollinger Bands Strategy and Backtesting

Prepared by Samalie Piwan   
Email : spiwan@andrew.cmu.edu

## 1. Backtest Implementation

Load the libraries to be used in the project

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
import seaborn as sns
import yfinance as yf
import requests
from io import StringIO
from backtesting import Backtest, Strategy
import backtesting
from tabulate import tabulate
import csv
from datetime import date
import os
backtesting.set_bokeh_output(notebook=False)

C:\Users\Sama\anaconda3\Lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

Declare the Simple Moving Average Window  and Standard Deviation to run the first version of the strategy with

In [3]:
# Declare the variables that will be used globally as SMA window, Short Period and Long Period / Standard Deviation
sma_window = 20
std = 2

Set the fiat currency pair to run the first version of the strategy on

In [3]:
from_symbol = "EUR"
to_symbol = "USD"
interval = "60min"
api_key = "CV4O3KUIMS9TVCLR"

### 1.1 Strategy and Returns on Annual FX data  - Jan 2005 to Dec 2013

Define the Indicator(Bollinger Bands) and Buy / Sell Strategy that the backtesting.py library will optimize

In [25]:
def indicator(data, window, std_dev):
    bbands = ta.bbands(close = data.Close.s, length = window, std = std_dev)
    return bbands.to_numpy().T[:3]

class BollingerBandsStrategy(Strategy):
        
    sma_window = 20
    std = 2
    
    def init(self):
        self.bbands = self.I(indicator, self.data, self.sma_window, self.std)
        
    def next(self):
        lower_band = self.bbands[0]
        upper_band = self.bbands[2]

        if self.position:
            if self.data.Close[-1] > upper_band[-1]:
                self.position.close()
        else:
            if self.data.Close[-1] < lower_band[-1]:
                self.buy()

Create a list of the currency pairs

In [35]:
currency_pairs = ["EUR/USD", "GBP/USD", "USD/JPY", "AUD/USD", "USD/CAD", "USD/CHF", "NZD/USD", "EUR/JPY", "GBP/JPY", "EUR/GBP", "EUR/CHF", "USD/SGD", "AUD/JPY", "CAD/JPY", "GBP/AUD", "EUR/AUD"]

Load the data from the API

In [42]:
for pair in currency_pairs:
    symbols = pair.split('/')
    from_symbol = symbols[0].strip()
    to_symbol = symbols[1].strip()
    # Load the monthly data from the API
    monthly_url = f"https://www.alphavantage.co/query?function=FX_MONTHLY&from_symbol={from_symbol}&to_symbol={to_symbol}&apikey={api_key}&datatype=csv"
    #monthly_request = requests.get(monthly_url)

    #Convert the CSV to a pandas dataframe
    monthly_data = StringIO(monthly_request.text)
    csv_res_monthly_df = pd.read_csv(monthly_data)

    monthly_data_df = csv_res_monthly_df.copy()
    monthly_data_df['timestamp'] = pd.to_datetime(monthly_data_df['timestamp'])
    monthly_pricing_df = monthly_data_df.set_index('timestamp')
    monthly_pricing_df.sort_index(ascending=True, inplace=True)

    #Seperate the data into two sets, 2005 to 2013 and 2014 to 2022
    backtest_dataset = pd.DataFrame(monthly_pricing_df.loc['2005-01-01' : '2013-12-31'].copy())
    rerun_dataset = pd.DataFrame(monthly_pricing_df.loc['2014-01-01':'2022-12-31'].copy())

    backtest_dataset = backtest_dataset.rename(columns = {'open':'Open', 'high':'High', 'low':'Low', 'close':'Close'})
    rerun_dataset = rerun_dataset.rename(columns = {'open':'Open', 'high':'High', 'low':'Low', 'close':'Close'})
    
    backtest_filename = "backtest"+from_symbol+""+to_symbol+".csv"
    rerun_filename = "rerun"+from_symbol+""+to_symbol+".csv"

    backtest_dataset.to_csv("backtestdata/"+backtest_filename)
    rerun_dataset.to_csv("rerundata/"+rerun_filename)

Define the function that will run the backtests for the currency pair list above

In [83]:
backtest_results = []
backtest_dir = 'backtestdata'
files = os.listdir(backtest_dir)

def backtest_strategy(param_cash, param_commission):
    index = 0
    while index < len(files):
        try:
            filename = backtest_dir+"/"+files[index]
            symbol = filename[21:27]
            
            file_data = pd.read_csv(filename)
            
            data_df = file_data.copy()
            data_df.index = pd.DatetimeIndex(data_df['timestamp'])
            data_df.sort_index(ascending=True, inplace=True)
            
            bt = Backtest(data_df, BollingerBandsStrategy, cash = param_cash, commission = param_commission)
            stats = bt.run()
            backtest_results.append(list([symbol, stats]))
        
        except FileNotFoundError:
            print(f"File {filename} not found")
        
        except ValueError:
            break
        
        finally:
            index += 1
            
    return backtest_results

In [73]:
bulk_backtest_results = backtest_strategy(10000, 0.01)

In [122]:
for results in bulk_backtest_results:
    print(results[0])
    stats = results[1]
    pnl_backtest = np.cumsum(stats['_trades'])
    pnl_backtest_val = pnl_backtest['PnL'].iloc[-1]
    print(pnl_backtest_val)

AUDCHF
814.5816669999988
AUDJPY
464.3378999999998
AUDUSD
4214.624049999999
CADJPY
616.4949999999999
EURAUD
2291.7603790000003
EURCHF
-1659.056933999999
EURGBP
882.2615939999984
EURJPY
-1755.9894000000004
EURUSD
890.9703839999995
GBPAUD
-2847.3616600000014
GBPCHF
-3166.548480000001
GBPJPY
-3166.0562999999997
GBPUSD
-1077.4610129999996
NZDUSD
1429.4833279999998
USDCAD
2046.8150320000013
USDCHF
-2290.5909629999996
USDJPY
-1928.2620000000009


### 2.2 Cross validation

From our previous backtesting results, the optimal paramters are:

 - Optimal standard deviation : 1.4
 - Optimal SMA Window : 29

In [85]:
optmized_std = 1.4
optimized_window = 29

Run an instance of our Backtest class using  from section 2 using:
- The **'rerun'** files that contain data from 2014 to 2022
- Cash of $10,000
- Commission at 1%

In [104]:
class OptBollingerBandsStrategy(Strategy):
    
    std = optmized_std
    sma_window = optimized_window
    
    def init(self):
        self.bbands = self.I(indicator, self.data, self.sma_window, self.std)
        
    def next(self):
        lower_band = self.bbands[0]
        upper_band = self.bbands[2]

        if self.position:
            if self.data.Close[-1] > upper_band[-1]:
                self.position.close()
        else:
            if self.data.Close[-1] < lower_band[-1]:
                self.buy()

In [117]:
rerun_results = []
rerun_dir = 'rerundata'
files = os.listdir(rerun_dir)

def validate_strategy(param_cash, param_commission):
    index = 0
    while index < len(files):
        try:
            filename = rerun_dir+"/"+files[index]
            symbol = filename[15:21]
            
            file_data = pd.read_csv(filename)
            
            data_df = file_data.copy()
            data_df.index = pd.DatetimeIndex(data_df['timestamp'])
            data_df.sort_index(ascending=True, inplace=True)
            
            bt = Backtest(data_df, OptBollingerBandsStrategy, cash = param_cash, commission = param_commission)
            stats = bt.run()
            rerun_results.append(list([symbol, stats]))
        
        except FileNotFoundError:
            print(f"File {filename} not found")
        
        except ValueError:
            break
        
        finally:
            index += 1
            
    return rerun_results

In [118]:
bulk_validate_results = validate_strategy(10000, 0.01)

In [120]:
for result in bulk_validate_results:
    print(result[0])
    stats = result[1]
    pnl_rerun = np.cumsum(stats['_trades'])
    pnl_rerun_val = pnl_rerun['PnL'].iloc[-1]
    print(pnl_rerun_val)

AUDCHF
-882.9188419999988
AUDJPY
570.0467199999985
AUDUSD
1121.5464528000007
CADJPY
952.6324799999995
EURAUD
1082.7606537000001
EURCHF
-1035.9924848
EURGBP
451.77775489999846
EURJPY
2322.2707600000003
EURUSD
502.15364440000064
GBPAUD
657.3007135999995
GBPCHF
-1319.978268
GBPJPY
696.6432000000002
GBPUSD
-597.2395260000004
NZDUSD
221.77492310000022
USDCAD
1550.7800684000026
USDCHF
1011.2709577999985
USDJPY
727.7078399999991
USDMXN
-98.9501399999993
USDSGD
602.4581310000017
